In [ ]:
# 5.1.5
import numpy as np
def conv(x, kernel, pad, stride):
  new_x = np.zeros(x.shape + np.array([pad*2,pad*2]))
  if pad != 0:
    new_x[pad:-pad,pad:-pad] = x
  else:
    new_x = x
  # print(new_x)

  res = []
  n_row = kernel.shape[0]
  n_col = kernel.shape[1]
  for i in range(0, new_x.shape[0] - n_row + 1, stride):
    for j in range(0, new_x.shape[1] - n_col + 1, stride):
      # print(i, j)
      res.append((new_x[i:i+n_row, j:j+n_col] * kernel).sum())
      # print(new_x[i:i+n_row, j:j+n_col])
      # print((new_x[i:i+n_row, j:j+n_col] * kernel).sum())
  return res

# x = np.array([[1,1,1,0,0],[0,1,1,1,0],[0,0,1,1,1],[0,0,1,1,0],[0,1,1,0,0]])
# kernel = np.array([[1, 0, 1],[0, 1, 0],[1, 0, 1]])
# pad = 0
# stride = 1

x = np.array([[4, 2, -1],[-6, 0, 5],[3, 2, 2]])
kernel = np.array([[0, 1, 2],[1, -1, 0],[1, 0, -2]])
pad = 1
stride = 2

conv(x, kernel, pad, stride)

[-4.0, 3.0, -9.0, 5.0]

In [ ]:
# 5.2.2
import torch

# Создаем входной массив из двух изображений RGB 3*3
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])

print(input_images.shape)

def get_padding2d(input_images):
    pad = 1
    new_im = torch.zeros(list(torch.tensor(input_images.shape) + torch.tensor([0, 0, pad*2,pad*2])))
    # new_im = torch.zeros(input_images.shape + torch([0, 0, pad*2,pad*2]))
    for i in range(input_images.shape[0]):
      for j in range(input_images.shape[1]):
        new_im[i, j, pad:-pad,pad:-pad] = input_images[i, j, :, :]
    return new_im

get_padding2d(input_images)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# print(torch.allclose(get_padding2d(input_images), correct_padded_images))

torch.Size([2, 3, 3, 3])


tensor([[[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  2.,  0.],
          [ 0.,  3.,  4.,  5.,  0.],
          [ 0.,  6.,  7.,  8.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  9., 10., 11.,  0.],
          [ 0., 12., 13., 14.,  0.],
          [ 0., 15., 16., 17.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 18., 19., 20.,  0.],
          [ 0., 21., 22., 23.,  0.],
          [ 0., 24., 25., 26.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]],


        [[[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 27., 28., 29.,  0.],
          [ 0., 30., 31., 32.,  0.],
          [ 0., 33., 34., 35.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 36., 37., 38.,  0.],
          [ 0., 39., 40., 41.,  0.],
          [ 0., 42., 43., 44.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 45., 46., 

In [ ]:
# 5.2.3
import numpy as np


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):

  out_shape = [input_matrix_shape[0],
               out_channels,
              int((input_matrix_shape[2] + 2*padding - (kernel_size - 1) - 1) / stride + 1),
              int((input_matrix_shape[3] + 2*padding - (kernel_size - 1) - 1) / stride + 1)
               ]

  return out_shape

print(calc_out_shape(input_matrix_shape=[2, 3, 10, 10],
                   out_channels=10,
                   kernel_size=3,
                   stride=1,
                   padding=0))
print(np.array_equal(
    calc_out_shape(input_matrix_shape=[2, 3, 10, 10],
                   out_channels=10,
                   kernel_size=3,
                   stride=1,
                   padding=0),
    [2, 10, 8, 8]))

[2, 10, 8, 8]
True


In [ ]:
# 5.2.5
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)

def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)

def get_padding2d(input_images, pad):
    new_im = torch.zeros(list(torch.tensor(input_images.shape) + torch.tensor([0, 0, pad*2,pad*2])))
    for i in range(input_images.shape[0]):
      for j in range(input_images.shape[1]):
        if pad != 0:
          new_im[i, j, pad:-pad,pad:-pad] = input_images[i, j, :, :]
        else:
          new_im[i, j, :,:] =  input_images[i, j, :, :]
    return new_im

# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
        tens_with_pad = get_padding2d(input_tensor, pad=0)
        output_tensor = torch.zeros(calc_out_shape(input_tensor.shape, self.out_channels, self.kernel_size, self.stride, 0))

        n_row = self.kernel_size
        n_col = self.kernel_size

        conv_for_each_image = []
        for k in range(input_tensor.shape[0]): # for image 
          conv_for_each_filter = []
          for k_ker in range(self.out_channels): # for filter 
            conv_for_each_channel = []
            for s in range(self.in_channels): # for channel 
              conv_per_row = []
              for i in range(0, tens_with_pad.shape[2] - n_row + 1, self.stride):
                conv_per_col = []
                for j in range(0, tens_with_pad.shape[3] - n_col + 1, self.stride):
                  conv_per_col.append((tens_with_pad[k, s, i:i+n_row, j:j+n_col] * self.kernel[k_ker, s, :, :]).sum())
                conv_per_row.append(conv_per_col)
              conv_for_each_channel.append(conv_per_row) 
            conv_for_each_channel = torch.tensor(conv_for_each_channel)
            conv_squesed_to_one_channel = conv_for_each_channel.sum(dim=0, keepdim=True).tolist()[0]
            conv_for_each_filter.append(conv_squesed_to_one_channel)
          conv_for_each_image.append(conv_for_each_filter)
        
        conv_for_each_image = torch.tensor(conv_for_each_image)
        return conv_for_each_image

# Корректность реализации определится в сравнении со стандартным слоем из pytorch.
# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
# print(test_conv2d_layer(Conv2dLoop))

True


In [ ]:
# 5.2.6
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_counqt, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)
    
    print('-------------')
    print(conv2d_out.shape)
    print(conv2d_out)
    print('-------------')

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)

class Conv2dMatrix(ABCConv2d):
    # Функция преобразование кернела в матрицу нужного вида.
    def _unsqueeze_kernel(self, torch_input, output_height, output_width):
        print(torch_input.shape)
        m = torch.nn.ConstantPad1d((0, 1, 0, 1), 0)
        kernel_with_zeros = m(self.kernel)
        return kernel_with_zeros.flatten().unsqueeze(dim=0)

    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        kernel_unsqueezed = self._unsqueeze_kernel(torch_input, output_height, output_width)
        result = kernel_unsqueezed @ torch_input.view((batch_size, -1)).permute(1, 0)
        return result.permute(1, 0).view((batch_size, self.out_channels,
                                          output_height, output_width))

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dMatrix))

-------------
torch.Size([2, 1, 1, 1])
tensor([[[[ 5252.]]],


        [[[12596.]]]], grad_fn=<MkldnnConvolutionBackward0>)
-------------
torch.Size([2, 3, 4, 4])
True


In [ ]:
# 5.2.7
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


def create_and_call_conv2d_layer(conv2d_layer_class, stride, kernel, input_matrix):
    out_channels = kernel.shape[0]
    in_channels = kernel.shape[1]
    kernel_size = kernel.shape[2]

    layer = conv2d_layer_class(in_channels, out_channels, kernel_size, stride)
    layer.set_kernel(kernel)

    return layer(input_matrix)


class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)


def test_conv2d_layer(conv2d_layer_class, batch_size=2,
                      input_height=4, input_width=4, stride=2):
    kernel = torch.tensor(
                      [[[[0., 1, 0],
                         [1,  2, 1],
                         [0,  1, 0]],

                        [[1, 2, 1],
                         [0, 3, 3],
                         [0, 1, 10]],

                        [[10, 11, 12],
                         [13, 14, 15],
                         [16, 17, 18]]]])

    in_channels = kernel.shape[1]

    input_tensor = torch.arange(0, batch_size * in_channels *
                                input_height * input_width,
                                out=torch.FloatTensor()) \
        .reshape(batch_size, in_channels, input_height, input_width)

    custom_conv2d_out = create_and_call_conv2d_layer(
        conv2d_layer_class, stride, kernel, input_tensor)
    conv2d_out = create_and_call_conv2d_layer(
        Conv2d, stride, kernel, input_tensor)

    return torch.allclose(custom_conv2d_out, conv2d_out) \
             and (custom_conv2d_out.shape == conv2d_out.shape)


class Conv2dMatrixV2(ABCConv2d):
    # Функция преобразования кернела в нужный формат.
    def _convert_kernel(self):
        converted_kernel = self.kernel.flatten().unsqueeze(0)
        return converted_kernel

    # Функция преобразования входа в нужный формат.
    def _convert_input(self, torch_input, output_height, output_width):
        batch_size, out_channels, output_height, output_width = calc_out_shape(
            torch_input.shape, self.out_channels, self.kernel_size, 
            self.stride, padding=0)
        
        out_dim = [batch_size, out_channels, output_height, output_width]
        list_of_flatten_img = []
        for k in range(batch_size):
            image = torch_input[k]
            for l in range(out_channels):
                for i in range(0, output_height, self.stride):
                    for j in range(0, output_height, self.stride):
                        sl = image[:, i:i+self.kernel_size, j:j+self.kernel_size]
                        list_of_flatten_img.append(sl.flatten())
        list_of_flatten_img = torch.stack(list_of_flatten_img).transpose(0,1)
        return list_of_flatten_img


    def __call__(self, torch_input):
        batch_size, out_channels, output_height, output_width\
            = calc_out_shape(
                input_matrix_shape=torch_input.shape,
                out_channels=self.kernel.shape[0],
                kernel_size=self.kernel.shape[2],
                stride=self.stride,
                padding=0)

        converted_kernel = self._convert_kernel()
        converted_input = self._convert_input(torch_input, output_height, output_width)

        conv2d_out_alternative_matrix_v2 = converted_kernel @ converted_input
        return conv2d_out_alternative_matrix_v2.transpose(1,0).view(torch_input.shape[0],
                                                     self.out_channels, 
                                                     output_height,
                                                     output_width)

# Проверка происходит автоматически вызовом следующего кода
# (раскомментируйте для самостоятельной проверки,
#  в коде для сдачи задания должно быть закомментировано):
print(test_conv2d_layer(Conv2dMatrixV2))

tensor([[[[ 0.,  1.,  0.],
          [ 1.,  2.,  1.],
          [ 0.,  1.,  0.]],

         [[ 1.,  2.,  1.],
          [ 0.,  3.,  3.],
          [ 0.,  1., 10.]],

         [[10., 11., 12.],
          [13., 14., 15.],
          [16., 17., 18.]]]])
tensor([[ 0.,  1.,  0.,  1.,  2.,  1.,  0.,  1.,  0.,  1.,  2.,  1.,  0.,  3.,
          3.,  0.,  1., 10., 10., 11., 12., 13., 14., 15., 16., 17., 18.]])
tensor([[ 0., 48.],
        [ 1., 49.],
        [ 2., 50.],
        [ 4., 52.],
        [ 5., 53.],
        [ 6., 54.],
        [ 8., 56.],
        [ 9., 57.],
        [10., 58.],
        [16., 64.],
        [17., 65.],
        [18., 66.],
        [20., 68.],
        [21., 69.],
        [22., 70.],
        [24., 72.],
        [25., 73.],
        [26., 74.],
        [32., 80.],
        [33., 81.],
        [34., 82.],
        [36., 84.],
        [37., 85.],
        [38., 86.],
        [40., 88.],
        [41., 89.],
        [42., 90.]])
True


In [ ]:
# 5.6.10
import torch

N = 4
C = 3
C_out = 10
H = 8
W = 16

x = torch.ones((N, C, H, W))

# torch.Size([4, 10, 8, 16])
out1 = torch.nn.Conv2d(C, C_out, kernel_size=(3, 3), padding=1)(x)
# print(out1.shape) # для самопроверки

# torch.Size([4, 10, 8, 16])
out2 = torch.nn.Conv2d(C, C_out, kernel_size=(5, 5), padding=2)(x)
# print(out2.shape) # для самопроверки

# torch.Size([4, 10, 8, 16])
out3 = torch.nn.Conv2d(C, C_out, kernel_size=(7, 7), padding=3)(x)
# print(out3.shape) # для самопроверки

# torch.Size([4, 10, 8, 16])
out4 = torch.nn.Conv2d(C, C_out, kernel_size=(9, 9), padding=4)(x)
# print(out4.shape) # для самопроверки

# # torch.Size([4, 10, 8, 16])
out5 = torch.nn.Conv2d(C, C_out, kernel_size=(3, 5), padding=(1,2))(x)
# print(out5.shape) # для самопроверки

# # torch.Size([4, 10, 22, 30])
out6 = torch.nn.Conv2d(C, C_out, kernel_size=(3, 3), padding=8)(x)
# print(out6.shape) # для самопроверки

# # torch.Size([4, 10, 7, 15])
out7 = torch.nn.Conv2d(C, C_out, kernel_size=(4, 4), padding=1)(x)
# print(out7.shape) # для самопроверки

# # torch.Size([4, 10, 9, 17])
out8 = torch.nn.Conv2d(C, C_out, kernel_size=(2, 2), padding=1)(x)
# print(out8.shape) # для самопроверки

torch.Size([4, 10, 8, 16])
torch.Size([4, 10, 8, 16])
torch.Size([4, 10, 8, 16])
torch.Size([4, 10, 8, 16])
torch.Size([4, 10, 8, 16])
torch.Size([4, 10, 22, 30])
torch.Size([4, 10, 7, 15])
torch.Size([4, 10, 9, 17])
